In [14]:
import pandas as pd
import csv

In [3]:
df = pd.read_csv('filter tableau.csv')

In [67]:
df.drop(['description', 'city', 'type_bussiness'], axis=1).to_csv("establishments.csv",quoting = csv.QUOTE_NONNUMERIC, quotechar = '"',)

In [24]:
df.description = df.description.str.replace('\r\n\                ', '')

In [30]:
df.description = df.description.str.replace('\r', ' ')

In [37]:
df.drop(['description', 'city', 'type_bussiness'], axis=1)[:20]

,id,values_filter,filter,bill,chain,chain_place,count_reviews,metro,name,rating,...,reviews,change_mobility,pandemic,district,district_n,branch,exist_metro,count_cusine,values_filter_code,bill_str
0,38020,европейская,Кухня,1000.0,NaN,NaN,131 отзыв,Гостиный Двор,Квартира,9.2,...,131.0,0,1.0,Центральный район,18,0,1,2,33,от 700р. до 1249р.
1,38020,бургеры,Блюдо,1000.0,NaN,NaN,131 отзыв,Гостиный Двор,Квартира,9.2,...,131.0,0,1.0,Центральный район,18,0,1,2,18,от 700р. до 1249р.
2,42499,рыба и морепродукты,Блюдо,1500.0,NaN,NaN,19 отзывов,Адмиралтейская,Находка,8.5,...,19.0,1,1.0,Адмиралтейский район,1,0,1,3,89,от 1250 до 2499р.
3,42499,устрицы,Блюдо,1500.0,NaN,NaN,19 отзывов,Адмиралтейская,Находка,8.5,...,19.0,1,1.0,Адмиралтейский район,1,0,1,3,110,от 1250 до 2499р.
4,42499,крабы,Блюдо,1500.0,NaN,NaN,19 отзывов,Адмиралтейская,Находка,8.5,...,19.0,1,1.0,Адмиралтейский район,1,0,1,3,62,от 1250 до 2499р.
5,39613,европейская,Кухня,1800.0,NaN,NaN,50 отзывов,Приморская,ДачА,9.1,...,50.0,0,1.0,Василеостровский район,2,0,1,1,33,от 1250 до 2499р.
6,43598,вегетарианская,Кухня,600.0,NaN,NaN,12 отзывов,Маяковская,Healthy Conscience,8.4,...,12.0,1,1.0,Центральный район,18,0,1,1,19,от 301р. до 699р.
7,42667,европейская,Кухня,900.0,NaN,NaN,10 отзывов,Гостиный Двор,Банкет фуршет,9.6,...,10.0,0,1.0,Центральный район,18,0,1,1,33,от 700р. до 1249р.
8,40103,смешанная,Кухня,1000.0,NaN,NaN,8 отзывов,Озерки,Maestro,10.0,...,8.0,0,1.0,Выборгский район,3,0,1,2,96,от 700р. до 1249р.
9,40103,европейская,Кухня,1000.0,NaN,NaN,8 отзывов,Озерки,Maestro,10.0,...,8.0,0,1.0,Выборгский район,3,0,1,2,33,от 700р. до 1249р.


In [44]:
df['coords'] = list(map(lambda x, y: [x,y], df.lat, df.lng))

In [55]:
import pickle
from urllib.request import urlopen
import numpy as np

rfc = pickle.load(urlopen('https://d5drokgciqre30et6qav.apigw.yandexcloud.net/model_covid.pkl'))
rfc2 = pickle.load(urlopen('https://d5drokgciqre30et6qav.apigw.yandexcloud.net/model_norm.pkl'))
scaler = pickle.load(urlopen('https://d5drokgciqre30et6qav.apigw.yandexcloud.net/scaler_covid.pkl'))
scaler2 = pickle.load(urlopen('https://d5drokgciqre30et6qav.apigw.yandexcloud.net/scaler_norm.pkl'))

def get_prediction_pandemic(bill, mobility, change_mobility, district_n, branch, exist_metro, count_cusine, 
                     values_filter_code):
    X = np.column_stack([bill, mobility, change_mobility, district_n, branch,exist_metro, count_cusine, 
                         values_filter_code])
    X = scaler.transform(X)
    return str(int(round(rfc.predict_proba(X)[0][1], 2)*100))+'%'

def get_prediction(bill, mobility, district_n, branch, exist_metro, count_cusine, values_filter_code):
    X = np.column_stack([bill, mobility, district_n, branch,exist_metro, count_cusine, values_filter_code])
    X = scaler2.transform(X)
    return str(int(round(rfc2.predict_proba(X)[0][1], 2)*100))+'%'


In [56]:
bill= 1208
mobility = 0
change_mobility = 0
district_n = 1
branch = 0
exist_metro = 0
count_cusine = 1 
values_filter_code = 85
get_prediction(bill, mobility, district_n, branch, exist_metro, count_cusine, values_filter_code),\
get_prediction_pandemic(bill, mobility, change_mobility, district_n, branch, exist_metro, count_cusine, 
                     values_filter_code)

('51%', '35%')

In [57]:
import json
file = json.load(open('test.json', 'rb'))

In [65]:
file['event']['path']

'/get_prediction_pandemic'

In [89]:
np.mean([2000+900+300,2700+1700+500,2500+1000+100])
# np.mean([300, 500, 100])

3900.0